## Optimizing LSTM using Genetic Algorithm 

#### Importing required packages

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

from keras.layers import LSTM, Input, Dense
from keras.models import Sequential
from keras.layers import *
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
import time,math

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray
import operator
import random


np.random.seed(1120)
import warnings
#warnings.filterwarnings('ignore')

Using TensorFlow backend.
C:\Users\Ali\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


### France

In [2]:
# France
#df = pd.read_excel('LSTM.xlsx', usecols=[1], nrows=50000)
df = pd.read_excel('LSTM.xlsx', usecols=[1]) # Full Model
#
#df=df[0:50000]
#df=df[0:2500]
df.shape

(157822, 1)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157822 entries, 0 to 157821
Data columns (total 1 columns):
Consumption    157822 non-null int64
dtypes: int64(1)
memory usage: 1.2 MB


### Scaling

In [3]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(df)
df=pd.DataFrame(dataset)
df.columns=['Consumption']
df.head()

C:\Users\Ali\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:334: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,Consumption
0,0.476205
1,0.479645
2,0.479534
3,0.478346
4,0.451795


## Defining Functions

In [4]:
def train_evaluate(ga_individual_solution):  
    df1= df.copy(deep=True)
    print('Dataset Shape',df1.shape)

    # Decode GA solution to integer for window_size and num_units
    
    # Parameter 1
    no_wins = ga_individual_solution[0:2]
    no_wins = int(''.join(str(x) for x in no_wins), base=2)
    
    if no_wins==0:no_wins+=1
    elif no_wins==1:no_wins+=1  
    elif no_wins==2:no_wins+=1
    elif no_wins==3:no_wins+=1 
    
    print('Number of Windows are:\n',no_wins)
    
    # Parameter 2
    # Windows starting points
    s1=BitArray(ga_individual_solution[2:9]).uint #7 bits
    s2=BitArray(ga_individual_solution[9:18]).uint # 9 bits
    s3=BitArray(ga_individual_solution[18:28]).uint # 10 bits
    s4=BitArray(ga_individual_solution[28:39]).uint # 11 bits
    
    #  Windows length, max=31
    l1= BitArray(ga_individual_solution[39:44]).uint
    l2= BitArray(ga_individual_solution[44:49]).uint
    l3= BitArray(ga_individual_solution[49:54]).uint
    l4= BitArray(ga_individual_solution[54:59]).uint
    
    # Batch Size
    batch_size = BitArray(ga_individual_solution[59:66]).uint
    batch_size+=10
    print('Batch Size is:',batch_size)
    
    act = ga_individual_solution[66:68]
    act = int(''.join(str(x) for x in act), base=2)
    
    opt = ga_individual_solution[68:70]
    opt = int(''.join(str(x) for x in opt), base=2)
    
    # Activation Decode
    if act==0:act='tanh'
    elif act==1:act='relu'
    elif act==2:act='selu'
    else:act='elu'         
        
    #  Optimizer Decode
    if opt==0:opt='sgd'
    elif opt==1:opt='RMSprop'
    elif opt==2:opt='Adamax'
    else:opt='Adam'     
        
    length=[l1,l2,l3,l4]
    new_list = [n+1 if n==0 else n for n in length]
    l1,l2,l3,l4=new_list[0:4]
    #window_length=max(new_list)

    # Avoid Overlapping 1 & 2
    
    if s2-(s1+l1) <=-10:
        print('Overlap 1 and 2')
        return 0.80,

        #1 & 3
    elif s3-(s1+l1) <=-10:
        print('Overlap 1 and 3')
        return 0.80,

        # 1 & 4
    elif s4-(s1+l1)<=-10:
        print('Overlap 1 and 4')
        return 0.80,

    elif s3-(s2+l2) <=-10:
        print('Overlap 2 and 3')
        return 0.80,
              
    elif s4-(s2+l2) <=-10:
        print('Overlap 2 and 4')
        return 0.80,

    elif s4-(s3+l3) <=-10:
        print('Overlap 3 and 4')
        return 0.80,

    else:
        print('No Overlap')
    
    print('Windows length are:\n',l1,l2,l3,l4)
        
    print('1st Window Start:',s1)
    print('2nd Window Start:',s2)
    print('3rd Window Start:',s3)
    print('4th Window Start:',s4) 
    
    print('Activation Function is:',act)
    print('Optimizer is:',opt)
     
    # Return fitness score of 100 if window_size or num_unit is zero
    if batch_size==0:
        return 1, 
    
    if no_wins==1:
        lag=np.arange(s1,s1+l1,1)
        window_length=l1
        print('Only Lag is\n',lag)
        
    elif no_wins==2:
        lag1=np.arange(s1,s1+l1,1).tolist()
        lag2=np.arange(s2,s2+l2,1).tolist()
        lag=[lag1,lag2]
        lag = pad_sequences(lag, padding='post',value=1)
        lag = [item for sublist in lag for item in sublist]
        window_length=max(l1,l2)
        print('Two Lags are\n',lag)

    elif no_wins==3:
        lag1=np.arange(s1,s1+l1,1).tolist()
        lag2=np.arange(s2,s2+l2,1).tolist()
        lag3=np.arange(s3,s3+l3,1).tolist()
        lag=[lag1,lag2,lag3]
        lag = pad_sequences(lag, padding='post',value=1)
        lag = [item for sublist in lag for item in sublist]
        print('Three Lags are\n',lag)
        window_length=max(l1,l2,l3)

    elif no_wins==4:
        lag1=np.arange(s1,s1+l1,1).tolist()
        lag2=np.arange(s2,s2+l2,1).tolist()
        lag3=np.arange(s3,s3+l3,1).tolist()
        lag4=np.arange(s4,s4+l4,1).tolist()
        lag=[lag1,lag2,lag3,lag4]
        lag = pad_sequences(lag, padding='post',value=1)
        lag = [item for sublist in lag for item in sublist]
        window_length=max(l1,l2,l3,l4)
        print('Four Lags are\n',lag)
      
    print('Lags length:\n', len(lag))
    print('Length Used:\n', window_length)
    for col in df1.columns:
        for idx, l in enumerate(lag):
            df1.loc[:,col+"_"+str(idx)] = df1[col].shift(l)
           
    df1.dropna(how='any',inplace=True)
    dataset = df1.values # Converted dataframe to numpy ndarray
    dataset = dataset.astype('float32')
    print('New Dataset shape after Lags:',dataset.shape)
    
    # split into train and test sets
    train_size = int(len(dataset) * 0.80)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    print('Training and test data length',len(train), len(test))
    
    trainX=train[:,1:]
    trainY=train[:,0]
    testX=test[:,1:]
    testY=test[:,0]
    
    print('trainX.shape[0]: ',trainX.shape[0])
    print('window_length: ',window_length) 
    print('no_wins: ',no_wins)
    
    print('trainX shape:\n',trainX.shape)
    
    #LSTM
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0],window_length,(no_wins)))
    testX = np.reshape(testX, (testX.shape[0],window_length,(no_wins)))
    print('Train X shape after np.reshape',trainX.shape)
    print('Test X shape after np.reshape',testX.shape)
    print('Train Y Shape',trainY.shape)
    print('Test Y Shape',testY.shape)
      
    # Train LSTM model and predict on validation set
    #inputs = Input(shape=(window_size,1))
    model = Sequential() # New Instance of Model Object
    model.add(LSTM(50, input_shape=(window_length,(no_wins)))) 
    #model.add(Dense(10, activation=act))
    model.add(Dense(50, activation=act))
    model.add(Dense(40, activation=act))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=opt,loss='mean_squared_error')
    #model.fit(trainX, trainY, epochs=1, batch_size=10,shuffle=True)
    
    start = time.time()
    model.fit(trainX, trainY, epochs=70, shuffle=False,batch_size=batch_size,
              validation_data=(testX, testY), 
              callbacks=[EarlyStopping(monitor='val_loss', patience=20)], verbose=1)
    
    y_pred = model.predict(testX)
    end = time.time()
    
    # Calculate the RMSE score as fitness score for GA
    rmse = np.sqrt(mean_squared_error(testY, y_pred))
    print('Validation RMSE: ', rmse,'\n')
    print ("Model took %0.2f seconds to train"%(end - start))
    return rmse,

In [5]:
# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0

creator.create('FitnessMin', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMin)

In [6]:
# self
#population_size = 20
#num_generations = 30

population_size = 10
num_generations = 30

#gene_length = 69
gene_length = 70


toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1) #Generates single RAND
toolbox.register('individual', tools.initRepeat, creator.Individual,
                 toolbox.attr_bool, n=gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

In [7]:
print(len(toolbox.individual()))
print(len(toolbox.population(population_size)))

70
10


In [9]:
'''population_size = 2
num_generations = 3
gene_length = 24


toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5) #Generates single bernoulli
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n=gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)'''

"population_size = 2\nnum_generations = 3\ngene_length = 24\n\n\ntoolbox = base.Toolbox()\ntoolbox.register('binary', bernoulli.rvs, 0.5) #Generates single bernoulli\ntoolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, n=gene_length)\ntoolbox.register('population', tools.initRepeat, list , toolbox.individual)"

## Algorithm

*deap.algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen[, stats, halloffame, verbose])*

In [8]:
# Register the crossover operator
toolbox.register('mate', tools.cxUniform,indpb= 0.6)

# Register a mutation operator
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.1)

# Operator for selecting individuals for breeding
#toolbox.register('select', tools.selRoulette)
toolbox.register('select',tools.selTournament,tournsize=4)

# Register the evaluation function
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)

In [9]:
from keras.callbacks import EarlyStopping
import time

In [10]:
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.1, ngen = num_generations,
                                       stats=stats, halloffame=hof, verbose=True)

Dataset Shape (50000, 1)
Number of Windows are:
 1
Batch Size is: 59
Overlap 2 and 3
Dataset Shape (50000, 1)
Number of Windows are:
 4
Batch Size is: 112
No Overlap
Windows length are:
 17 30 20 1
1st Window Start: 43
2nd Window Start: 140
3rd Window Start: 999
4th Window Start: 1351
Activation Function is: elu
Optimizer is: Adamax
Four Lags are
 [43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1351, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Lags length:
 120
Length Used:
 30
New Dataset shape after Lags: (48649, 121)
Training and test data length 38919 9730
trainX.shape[0]:  38919
window_

39970/39970 [==============================] - 14s 348us/step - loss: 0.0130 - val_loss: 0.0211
Epoch 19/70
39970/39970 [==============================] - 14s 346us/step - loss: 0.0130 - val_loss: 0.0210
Epoch 20/70
39970/39970 [==============================] - 14s 347us/step - loss: 0.0130 - val_loss: 0.0210
Epoch 21/70
39970/39970 [==============================] - 14s 346us/step - loss: 0.0129 - val_loss: 0.0209
Epoch 22/70
39970/39970 [==============================] - 14s 347us/step - loss: 0.0129 - val_loss: 0.0209
Epoch 23/70
39970/39970 [==============================] - 14s 345us/step - loss: 0.0129 - val_loss: 0.0208
Epoch 24/70
39970/39970 [==============================] - 14s 346us/step - loss: 0.0129 - val_loss: 0.0208
Epoch 25/70
39970/39970 [==============================] - 14s 346us/step - loss: 0.0129 - val_loss: 0.0207
Epoch 26/70
39970/39970 [==============================] - 14s 348us/step - loss: 0.0129 - val_loss: 0.0207
Epoch 27/70
39970/39970 [===============

39894/39894 [==============================] - 8s 200us/step - loss: 0.0146 - val_loss: 0.0405
Epoch 16/70
39894/39894 [==============================] - 8s 199us/step - loss: 0.0146 - val_loss: 0.0406
Epoch 17/70
39894/39894 [==============================] - 8s 199us/step - loss: 0.0146 - val_loss: 0.0407
Epoch 18/70
39894/39894 [==============================] - 8s 199us/step - loss: 0.0147 - val_loss: 0.0408
Epoch 19/70
39894/39894 [==============================] - 8s 200us/step - loss: 0.0147 - val_loss: 0.0409
Epoch 20/70
39894/39894 [==============================] - 8s 200us/step - loss: 0.0147 - val_loss: 0.0409
Epoch 21/70
39894/39894 [==============================] - 8s 199us/step - loss: 0.0148 - val_loss: 0.0410
Validation RMSE:  0.20251024 

Model took 170.12 seconds to train
1  	8     	0.610301	0.290175	0.134017 	0.8
Dataset Shape (50000, 1)
Number of Windows are:
 1
Batch Size is: 92
Overlap 1 and 2
Dataset Shape (50000, 1)
Number of Windows are:
 1
Batch Size is: 68


Epoch 59/70
39971/39971 [==============================] - 14s 339us/step - loss: 0.0024 - val_loss: 0.0112
Epoch 60/70
39971/39971 [==============================] - 14s 340us/step - loss: 0.0024 - val_loss: 0.0108
Epoch 61/70
39971/39971 [==============================] - 14s 341us/step - loss: 0.0024 - val_loss: 0.0122
Epoch 62/70
39971/39971 [==============================] - 14s 339us/step - loss: 0.0024 - val_loss: 0.0112
Epoch 63/70
39971/39971 [==============================] - 14s 339us/step - loss: 0.0023 - val_loss: 0.0125
Epoch 64/70
39971/39971 [==============================] - 14s 339us/step - loss: 0.0023 - val_loss: 0.0113
Epoch 65/70
39971/39971 [==============================] - 13s 337us/step - loss: 0.0023 - val_loss: 0.0111
Validation RMSE:  0.10539708 

Model took 887.95 seconds to train
2  	4     	0.399558	0.327712	0.0891916	0.8
Dataset Shape (50000, 1)
Number of Windows are:
 1
Batch Size is: 124
No Overlap
Windows length are:
 28 22 23 9
1st Window Start: 9
2n

39971/39971 [==============================] - 14s 339us/step - loss: 0.0036 - val_loss: 0.0090
Epoch 28/70
39971/39971 [==============================] - 13s 337us/step - loss: 0.0036 - val_loss: 0.0087
Epoch 29/70
39971/39971 [==============================] - 13s 338us/step - loss: 0.0035 - val_loss: 0.0085
Epoch 30/70
39971/39971 [==============================] - 14s 339us/step - loss: 0.0035 - val_loss: 0.0084
Epoch 31/70
39971/39971 [==============================] - 14s 342us/step - loss: 0.0034 - val_loss: 0.0081
Epoch 32/70
39971/39971 [==============================] - 14s 340us/step - loss: 0.0034 - val_loss: 0.0081
Epoch 33/70
39971/39971 [==============================] - 14s 341us/step - loss: 0.0033 - val_loss: 0.0087
Epoch 34/70
39971/39971 [==============================] - 13s 337us/step - loss: 0.0032 - val_loss: 0.0085
Epoch 35/70
39971/39971 [==============================] - 14s 341us/step - loss: 0.0032 - val_loss: 0.0085
Epoch 36/70
39971/39971 [===============

...

Epoch 69/70
39182/39182 [==============================] - 22s 556us/step - loss: 1.0442e-04 - val_loss: 9.0520e-05
Epoch 70/70
39182/39182 [==============================] - 22s 556us/step - loss: 1.0374e-04 - val_loss: 9.2122e-05
Validation RMSE:  0.009598017 

Model took 1560.81 seconds to train
Dataset Shape (50000, 1)
Number of Windows are:
 4
Batch Size is: 68
No Overlap
Windows length are:
 16 20 19 8
1st Window Start: 15
2nd Window Start: 502
3rd Window Start: 1004
4th Window Start: 1132
Activation Function is: relu
Optimizer is: RMSprop
Four Lags are
 [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 1, 1, 1, 1, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Lags length:
 80
Length Used:
 20
New Dataset shape aft

39182/39182 [==============================] - 21s 546us/step - loss: 3.0579e-04 - val_loss: 2.7082e-04
Epoch 12/70
39182/39182 [==============================] - 21s 535us/step - loss: 2.9739e-04 - val_loss: 2.7336e-04
Epoch 13/70
39182/39182 [==============================] - 21s 538us/step - loss: 2.8819e-04 - val_loss: 2.6696e-04
Epoch 14/70
39182/39182 [==============================] - 21s 540us/step - loss: 2.8037e-04 - val_loss: 2.4569e-04
Epoch 15/70
39182/39182 [==============================] - 21s 539us/step - loss: 2.7028e-04 - val_loss: 2.4783e-04
Epoch 16/70
39182/39182 [==============================] - 21s 536us/step - loss: 2.6295e-04 - val_loss: 2.2041e-04
Epoch 17/70
39182/39182 [==============================] - 21s 540us/step - loss: 2.5551e-04 - val_loss: 2.1710e-04
Epoch 18/70
39182/39182 [==============================] - 21s 536us/step - loss: 2.4887e-04 - val_loss: 1.9874e-04
Epoch 19/70
39182/39182 [==============================] - 21s 537us/step - loss: 2.

New Dataset shape after Lags: (48978, 61)
Training and test data length 39182 9796
trainX.shape[0]:  39182
window_length:  20
no_wins:  3
trainX shape:
 (39182, 60)
Train X shape after np.reshape (39182, 20, 3)
Test X shape after np.reshape (9796, 20, 3)
Train Y Shape (39182,)
Test Y Shape (9796,)
Train on 39182 samples, validate on 9796 samples
Epoch 1/70
39182/39182 [==============================] - 60s 2ms/step - loss: 0.0048 - val_loss: 0.0040
Epoch 2/70
39182/39182 [==============================] - 21s 543us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 3/70
39182/39182 [==============================] - 21s 541us/step - loss: 7.5984e-04 - val_loss: 7.5055e-04
Epoch 4/70
39182/39182 [==============================] - 21s 542us/step - loss: 5.8604e-04 - val_loss: 4.1268e-04
Epoch 5/70
39182/39182 [==============================] - 21s 543us/step - loss: 5.0676e-04 - val_loss: 3.2502e-04
Epoch 6/70
39182/39182 [==============================] - 21s 541us/step - loss: 4.5528e-04 - v

Epoch 69/70
39182/39182 [==============================] - 21s 542us/step - loss: 1.1719e-04 - val_loss: 8.1145e-05
Epoch 70/70
39182/39182 [==============================] - 21s 540us/step - loss: 1.1730e-04 - val_loss: 7.9841e-05
Validation RMSE:  0.008935389 

Model took 1541.91 seconds to train
Dataset Shape (50000, 1)
Number of Windows are:
 3
Batch Size is: 70
No Overlap
Windows length are:
 28 20 3 8
1st Window Start: 15
2nd Window Start: 502
3rd Window Start: 940
4th Window Start: 1388
Activation Function is: relu
Optimizer is: RMSprop
Three Lags are
 [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 1, 1, 1, 1, 1, 1, 1, 1, 940, 941, 942, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Lags length:
 84
Length Used:
 28
New Dataset shape after Lags: (49058, 85)
Training and test data length 3

39182/39182 [==============================] - 21s 540us/step - loss: 2.9125e-04 - val_loss: 2.1978e-04
Epoch 14/70
39182/39182 [==============================] - 21s 541us/step - loss: 2.8006e-04 - val_loss: 1.9766e-04
Epoch 15/70
39182/39182 [==============================] - 21s 540us/step - loss: 2.7005e-04 - val_loss: 1.7661e-04
Epoch 16/70
39182/39182 [==============================] - 21s 542us/step - loss: 2.6341e-04 - val_loss: 1.7359e-04
Epoch 17/70
39182/39182 [==============================] - 21s 539us/step - loss: 2.5759e-04 - val_loss: 1.7877e-04
Epoch 18/70
39182/39182 [==============================] - 21s 542us/step - loss: 2.5194e-04 - val_loss: 1.9116e-04
Epoch 19/70
39182/39182 [==============================] - 21s 542us/step - loss: 2.4264e-04 - val_loss: 1.8380e-04
Epoch 20/70
39182/39182 [==============================] - 21s 541us/step - loss: 2.3151e-04 - val_loss: 1.6362e-04
Epoch 21/70
39182/39182 [==============================] - 21s 543us/step - loss: 2.

### Print top N solutions

In [30]:
a=tools.selBest(population,k = 1)
a=np.asarray(a)
a

array([[1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 1, 0, 1]])

In [1]:
a=[1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
        1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 1, 0, 1]

In [13]:
a[68:70]

[0, 1]

In [33]:
len(a[0])

70

### Results

In [26]:
#Save Results
gen,avg,min1,max1,nevals, std = log.select('gen','avg','min','max','nevals','std')
df=pd.DataFrame({'Generation':gen, 'Average':avg, 'Minimum':min1, 'Maximum':max,
                 'N_Evaluations':nevals, 'Standard Dev':std})
df

,Generation,Average,Minimum,Maximum,N_Evaluations,Standard Dev
0,0,0.657151,0.021108,0.800000,10,0.287157
1,1,0.610301,0.134017,0.800000,8,0.290175
2,2,0.399558,0.089192,0.800000,4,0.327712
3,3,0.179253,0.089192,0.800000,4,0.207731
4,4,0.088026,0.011894,0.113052,5,0.026897
5,5,0.068270,0.011694,0.128148,6,0.045749
6,6,0.033974,0.011376,0.117924,8,0.041500
7,7,0.015175,0.011376,0.039375,6,0.008119
8,8,0.011713,0.008900,0.014074,8,0.001868
9,9,0.013215,0.008900,0.024492,9,0.004730


In [28]:
writer = pd.ExcelWriter('GA_Final_Res.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

## Plots

In [14]:
sns.set_style('white')
sns.set_context("paper", font_scale=1.2)  

plt.figure(figsize=(8,4))
#avg1,min1,max1=scaler.inverse_transform([avg1,min1,max1])

plt.plot(gen1,avg1,marker='s',linewidth=1,markersize=6,label='Average')
plt.plot(gen1,min1,marker='d',linewidth=1,markersize=6,label='Minimum')
#plt.plot(gen1,max1,marker='*',linewidth=1,markersize=8,label='Maximum')
#plt.tick_params(left=False, labelleft=True)
#plt.box(False)
plt.margins(0.2)


plt.xlabel('Generation Number',labelpad=15)
plt.ylabel('RMSE LSTM')
#plt.xlim(-1, 26)
plt.legend(loc='best');
sns.despine(left=True)
plt.tight_layout()
plt.savefig('Fig1.png', bbox_inches='tight', dpi=400) 
plt.show()

In [ ]:
# Box Plot
a,b,c=log.select('min','max','std')

#a,b,c=scaler.inverse_transform([a,b,c])

df=pd.DataFrame({'Minimum':a,'Maximum':b, 'Std':c})

plt.figure(figsize=(6,4))
sns.set_style('white')
sns.set_context("paper", font_scale=1.3)  
sns.boxplot(data=df, width=0.6,color='#1f77b4')
plt.xlabel('Statistics across Generations and Particles',labelpad=15)
plt.ylabel('RMSE LSTM')
sns.despine(left=True)
plt.tight_layout()
plt.savefig('Fig1.png', bbox_inches='tight', dpi=400) 
plt.show()

### Save Results

In [15]:
a,b,c,d,e=log.select('gen','avg','min','max','std')

#b,c,d,e=scaler.inverse_transform([b,c,d,e])
df=pd.DataFrame({'Generation':a,'Average':b, 'Minimum':c,'Maximum':d,'Std':e})
writer = pd.ExcelWriter('GA_Final__norm.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

## Full Model

In [4]:
df1=df.copy(deep=True)
df1.head(2)

,Consumption
0,0.476205
1,0.479645


In [5]:
#Lags LSTM
lag1=np.arange(1,18,1)
lag2=np.arange(509,526,1)
lag3=np.arange(1003,1020,1)
lag=np.concatenate([lag1,lag2,lag3])
print('The number of lags are:',lag.shape)
lag

The number of lags are: (51,)


array([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,   11,
         12,   13,   14,   15,   16,   17,  509,  510,  511,  512,  513,
        514,  515,  516,  517,  518,  519,  520,  521,  522,  523,  524,
        525, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012,
       1013, 1014, 1015, 1016, 1017, 1018, 1019])

In [6]:
for col in df1.columns:
        for idx, l in enumerate(lag):
            df1.loc[:,col+"_"+str(idx)] = df1[col].shift(l)
           
df1.dropna(how='any',inplace=True)
dataset = df1.values # Converted dataframe to numpy ndarray
dataset = dataset.astype('float32')
print('New Dataset shape after Lags:',dataset.shape)

New Dataset shape after Lags: (156803, 52)


In [7]:
# split into train and test sets
train_size = int(len(dataset) * 0.80)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print('Training and test data length',len(train), len(test))
    
trainX=train[:,1:]
trainY=train[:,0]
testX=test[:,1:]
testY=test[:,0]
    
print('trainX.shape[0]: ',trainX.shape[0])
print('trainX shape:\n',trainX.shape)

Training and test data length 125442 31361
trainX.shape[0]:  125442
trainX shape:
 (125442, 51)


In [8]:
#LSTM
# reshape input to be [samples, time steps, features]
#trainX = np.reshape(trainX, (trainX.shape[0],window_length,(no_wins)))
#testX = np.reshape(testX, (testX.shape[0],window_length,(no_wins)))

trainX = np.reshape(trainX, (trainX.shape[0],17,3))
testX = np.reshape(testX, (testX.shape[0],17,3))


print('Train X shape after np.reshape',trainX.shape)
print('Test X shape after np.reshape',testX.shape)
print('Train Y Shape',trainY.shape)
print('Test Y Shape',testY.shape)

Train X shape after np.reshape (125442, 17, 3)
Test X shape after np.reshape (31361, 17, 3)
Train Y Shape (125442,)
Test Y Shape (31361,)


In [9]:
# create and fit the LSTM network
model = Sequential() # New Instance of Model Object
model.add(LSTM(60, input_shape=(17,3)))
model.add(Dense(100, activation='relu'))
model.add(Dense(70, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='Adam')

Instructions for updating:
Colocations handled automatically by placer.


In [10]:
start = time.time()
hist =model.fit(trainX, trainY, epochs=120, shuffle=True,batch_size=125, validation_data=(testX, testY), 
                callbacks=[EarlyStopping(monitor='val_loss', patience=20)], verbose=1)

end = time.time()
# Training Phase
model.summary()

Instructions for updating:
Use tf.cast instead.
Train on 125442 samples, validate on 31361 samples
Epoch 1/120
125442/125442 [==============================] - 31s 247us/step - loss: 0.0033 - val_loss: 3.0076e-04
Epoch 2/120
125442/125442 [==============================] - 28s 220us/step - loss: 3.9085e-04 - val_loss: 2.7266e-04
Epoch 3/120
125442/125442 [==============================] - 28s 220us/step - loss: 3.4971e-04 - val_loss: 2.7059e-04
Epoch 4/120
125442/125442 [==============================] - 28s 220us/step - loss: 3.2310e-04 - val_loss: 3.1914e-04
Epoch 5/120
125442/125442 [==============================] - 28s 220us/step - loss: 2.8972e-04 - val_loss: 2.3260e-04
Epoch 6/120
125442/125442 [==============================] - 27s 219us/step - loss: 2.8970e-04 - val_loss: 3.5295e-04
Epoch 7/120
125442/125442 [==============================] - 28s 220us/step - loss: 2.8629e-04 - val_loss: 2.5629e-04
Epoch 8/120
125442/125442 [==============================] - 27s 219us/step - l

In [11]:
# Make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

In [12]:
# Invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [13]:
from sklearn.metrics import mean_absolute_error

In [14]:
# calculate root mean squared error
print('Mean Absolute Error Train:', mean_absolute_error(trainY[0], trainPredict[:,0]))

testScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score RMSE: %.2f RMSE' % (testScore))

from sklearn.metrics import mean_absolute_error
print('Mean Absolute Error Test:', mean_absolute_error(testY[0], testPredict[:,0]))
print('Mean Squared Error Test:',np.sqrt(mean_squared_error(testY[0], testPredict[:,0])))
#print('Mean Absolute Percentage Error:',MAPError(testY[0], testPredict[:,0]))

Mean Absolute Error Train: 242.16780533830598
Train Score RMSE: 323.49 RMSE
Mean Absolute Error Test: 240.9821425866052
Mean Squared Error Test: 321.2727546409887


In [17]:
trainY.shape, testY.shape

((1, 125442), (1, 31361))

In [22]:
(321.27/(np.mean(testY)))*100

0.6104836323543474